In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ultrasound_dataset import USDataset, Moco2TrainTransforms, Moco2EvalTransforms

from torchvision import transforms

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import SimpleITK as sitk
import numpy as np

import pickle

/work/jprieto/install/anaconda3/envs/us/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mount_point = '/work/jprieto/data/remote/GWH/Groups/FAMLI/Shared/C1_ML_Analysis'
df_test = pd.read_csv(os.path.join(mount_point, 'CSV_files', 'c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_test.csv'))

feat_fn = os.path.join(mount_point, 'test_output/ga3d/', 'v1/epoch=105-val_loss=179.60/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_test.pickle')
x, x_a, x_s, x_v, x_v_p = pickle.load(open(feat_fn, 'rb'))

In [4]:
df_test

,Unnamed: 0.1,Unnamed: 0,PID,PIDscan,PID_date,PID_date_time,study_dttm,content_dttm,path,file,...,fetal_anomaly,study_finding,study_lim,study_comments,is_anchor,ga_boe,ga_expert,ga_absdiff,ga_boe_w,is_ga_anchor
0,0,0,FAM-025-0453,FAM-025-0453-5,FAM-025-0453-5_20191106,FAM-025-0453-5_20191106_111051,2019-11-06 11:10:51,2019-11-06 11:34:44,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.347.59684.3586430.11...,...,0,us_none,0,NaN,0.0,255.726434,.,.,36.532348,0.0
1,1,1,FAM-025-0453,FAM-025-0453-5,FAM-025-0453-5_20191106,FAM-025-0453-5_20191106_111051,2019-11-06 11:10:51,2019-11-06 11:35:08,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.347.59708.1032022.11...,...,0,us_none,0,NaN,0.0,255.726434,.,.,36.532348,0.0
2,2,2,FAM-025-0453,FAM-025-0453-5,FAM-025-0453-5_20191106,FAM-025-0453-5_20191106_111051,2019-11-06 11:10:51,2019-11-06 11:35:38,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.347.59738.7503747.11...,...,0,us_none,0,NaN,0.0,255.726434,.,.,36.532348,0.0
3,3,3,FAM-025-0453,FAM-025-0453-5,FAM-025-0453-5_20191106,FAM-025-0453-5_20191106_111051,2019-11-06 11:10:51,2019-11-06 11:36:03,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.347.59763.7081666.11...,...,0,us_none,0,NaN,0.0,255.726434,.,.,36.532348,0.0
4,4,4,FAM-025-0453,FAM-025-0453-5,FAM-025-0453-5_20191106,FAM-025-0453-5_20191106_111051,2019-11-06 11:10:51,2019-11-06 11:36:30,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.347.59790.6585239.11...,...,0,us_none,0,NaN,0.0,255.726434,.,.,36.532348,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,18056,19183,VIL-0594,VIL-0594-3,VIL-0594-3_20190822,VIL-0594-3_20190822_094717,2019-08-22 09:47:17,2019-08-22 09:53:57,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.270.50037.2875334.11...,...,0,NaN,0,NaN,0.0,231.000000,241.30862,10.3086199999999,33.000000,0.0
3521,18057,19184,VIL-0594,VIL-0594-3,VIL-0594-3_20190822,VIL-0594-3_20190822_094717,2019-08-22 09:47:17,2019-08-22 09:54:14,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.270.50054.2647455.11...,...,0,NaN,0,NaN,0.0,231.000000,241.30862,10.3086199999999,33.000000,0.0
3522,18058,19185,VIL-0594,VIL-0594-3,VIL-0594-3_20190822,VIL-0594-3_20190822_094717,2019-08-22 09:47:17,2019-08-22 09:59:10,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.270.50350.3307258.11...,...,0,NaN,0,NaN,0.0,231.000000,241.30862,10.3086199999999,33.000000,0.0
3523,18059,19186,VIL-0594,VIL-0594-3,VIL-0594-3_20190822,VIL-0594-3_20190822_094717,2019-08-22 09:47:17,2019-08-22 09:59:40,W:\Groups\FAMLI\Restricted_access_data\Ultraso...,1.2.276.0.26.1.1.1.2.2019.270.50380.4456055.11...,...,0,NaN,0,NaN,0.0,231.000000,241.30862,10.3086199999999,33.000000,0.0


In [18]:
df_group = df_test.groupby('StudyID')
ga_boe = []

for key, dfg in df_group:
    ga_boe.append({'StudyID': key, 'ga_boe': float(dfg['ga_boe'].unique()[0])})


In [22]:
ga_boe = pd.DataFrame(ga_boe)
ga_boe = x.reshape(-1)

In [ ]:

img1, img2 = test_ds[np.random.randint(len(test_ds))]
img1 = img1.permute(1,2,0).numpy()
img2 = img2.permute(1,2,0).numpy()

# img1 = ((img1 - np.min(img1))/(np.max(img1) - np.min(img1)))*255
# img2 = ((img2 - np.min(img2))/(np.max(img2) - np.min(img2)))*255
img1 *= 255
img2 *= 255

fig = px.imshow(np.array([img1, img2]), facet_col=0, binary_string=True, height=512)        
fig

In [103]:

df_full = pd.read_csv(os.path.join(mount_point, 'CSV_files', 'c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_st_tt_morethan3_train.csv'))

feat_full_fn = os.path.join(mount_point, 'test_output/ga3d/st_tt_count3/v1/epoch=607-val_loss=65.28/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_st_tt_morethan3_train.pickle')
x, x_a, x_s, x_v, x_v_p = pickle.load(open(feat_full_fn, 'rb'))

In [113]:
ga_column = 'ga_boe'
id_column = 'StudyID'

df_groups = df_full.groupby(id_column)
df_g = []
for k in df_groups.groups.keys():
    df_group = df_groups.get_group(k)
    ga = float(df_group[ga_column].unique()[0])
    study_dttm = df_group['study_dttm'].unique()[0]
    Manufacturer = df_group['Manufacturer'].unique()[0]
    df_g.append({id_column: k, ga_column: ga, 'study_dttm': study_dttm, 'Manufacturer': Manufacturer})
df_g = pd.DataFrame(df_g)

In [115]:
df_g['pred'] = x.reshape(-1)
df_g['error'] = df_g['pred'] - df_g[ga_column]
df_g['abs'] = abs(df_g['error'])

In [116]:
df_g['abs'].describe()

count    2602.000000
mean        3.507704
std         3.021458
min         0.000593
25%         1.318365
50%         2.781659
75%         4.908624
max        33.072470
Name: abs, dtype: float64

In [126]:
folds = ["test_output/ga3d/fold0/epoch=533-val_loss=95.78/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_trainfold0_test.pickle",
"test_output/ga3d/fold1/epoch=656-val_loss=70.29/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_trainfold1_test.pickle",
"test_output/ga3d/fold2/epoch=696-val_loss=69.60/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_trainfold2_test.pickle",
"test_output/ga3d/fold3/epoch=648-val_loss=62.30/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_trainfold3_test.pickle",
"test_output/ga3d/fold4/epoch=462-val_loss=72.84/c1_volumes_ga_boe_20220811_masked_resampled_128_spc075_notna_trainfold4_test.pickle"]


In [139]:
x = []
x_a = []
x_s = []
x_v = []
x_v_p = []

df_folds = []

for f in folds:
    x_, x_a_, x_s_, x_v_, x_v_p_ = pickle.load(open(os.path.join(mount_point, f), 'rb'))
    
    x.append(x_)
    x_a.append(x_a)
    x_s.append(x_s_)
    x_v.append(x_v_)
    x_v_p.append(x_v_p_)
    
    df_test = pd.read_csv(os.path.join(mount_point, 'CSV_files', os.path.basename(f).replace(".pickle", ".csv")))
    
    df_groups = df_test.groupby(id_column)
    df_g = []
    for k in df_groups.groups.keys():
        df_group = df_groups.get_group(k)
        ga = float(df_group[ga_column].unique()[0])
        study_dttm = df_group['study_dttm'].unique()[0]
        Manufacturer = df_group['Manufacturer'].unique()[0]
        df_g.append({id_column: k, ga_column: ga, 'study_dttm': study_dttm, 'Manufacturer': Manufacturer})
        
    df_g = pd.DataFrame(df_g)
    df_g['pred'] = x_.reshape(-1)
    
    df_folds.append(df_g)


In [141]:
df_folds = pd.concat(df_folds)

In [142]:
df_folds

,StudyID,ga_boe,study_dttm,Manufacturer,pred
0,FAM-025-0605-5,209.695665,2019-11-29 12:21:52,GE Healthcare,218.407700
1,FAM-025-0605-6,254.695665,2020-01-13 11:19:08,GE Healthcare,253.376862
2,FAM-025-0650-2,237.731427,2019-11-18 13:49:51,GE Healthcare,241.041504
3,FAM-025-0659-4,169.522678,2019-11-06 13:23:09,GE Healthcare,169.788361
4,FAM-025-0659-5,197.522678,2019-12-04 11:37:44,GE Healthcare,191.629028
...,...,...,...,...,...
642,VIL-0317-2,170.000000,2019-04-22 15:06:05,GE Healthcare Austria GmbH & Co OG,173.875992
643,VIL-0317-3,198.000000,2019-05-20 10:49:03,GE Healthcare Austria GmbH & Co OG,204.365631
644,VIL-0317-4,228.000000,2019-06-19 14:53:25,GE Healthcare Austria GmbH & Co OG,238.170456
645,VIL-0414-3,128.000000,2019-05-10 14:24:38,GE Healthcare Austria GmbH & Co OG,128.967270
